In [1]:
import pandas as pd
import numpy as np
import pymysql
import warnings
warnings.filterwarnings('ignore')

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
sql_black = '''
SELECT
id_card, mobile, add_time, cause
FROM db_credit.blacklist b
WHERE b.end_time is NULL or b.end_time>SYSDATE()
'''
# b.end_time is NULL: 永久黑名单（没有结束时间）
# b.end_time>SYSDATE(): 临时黑名单但尚未到期（结束时间大于当前系统时间）
df_black = query(sql_black)

In [4]:
df_black.loc[:, 'year'] = df_black.add_time.dt.year
df_black.loc[:, '添加日期'] = df_black.add_time.dt.date
# 命中黑名单项

# df_black.cause.drop_duplicates().to_excel('F:/black.xlsx', index=False)

In [5]:
def black(cause):
    if '年龄' in cause or '下单次数' in cause or '临时黑名单' in cause or '频繁' in cause:
        return '命中强拒加入临时黑名单'
    elif '虚假' in cause or '非本人' in cause or '中介' in cause or '第三方' in cause or '电话审核' in cause or '套现' in cause or '团伙' in cause:
        return '命中疑似欺诈强拒'
    elif '法院限高' in cause or '被执行人' in cause:
        return '命中法院限高/被执行人强拒'
    elif '纠纷' in cause or '案件' in cause:
        return '命中案件纠纷判定强拒'
    elif '特殊' in cause or '风险' in cause:
        return '命中特殊名单|地区|策略验证-高风险强拒'
    elif '行业黑名单' in cause:
        return '命中行业黑名单强拒'
    elif '客服' in cause or '添加' in cause:
        return '客服手动添加'
    elif '蚂蚁数控' in cause:
        return '蚂蚁数控强拒'
    else:
        return '其他'

In [6]:
df_black.loc[:, '命中项'] = df_black.apply(lambda x: black(x['cause']), axis=1)
# pd.pivot_table(df_black, values='id_card', index='year', columns='命中项', aggfunc='count').to_excel('F:/需求/总需求/黑名单命中项.xlsx') 

In [7]:
# df_black[['添加日期', 'id_card', 'mobile', '命中项']].to_excel('F:/需求/总需求/黑名单明细.xlsx', index=False)

In [8]:
# df_black_new = df_black.groupby(['year', 'id_card']).agg({'cause': 'count'})
# df_black_new = df_black_new.reset_index()
# df_black_new
# pd.pivot_table(df_black_new, values='id_card', index='year', columns='cause', aggfunc='count')